In [1]:
import math
import csv
import pvlib
import os
import sys
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import datetime

sys.path.insert(0, '../bifacialvf')

import bifacialvf   
from loadVFresults import loadVFresults

print "Loaded"


Loaded


In [3]:
compiledwritefiletitle="..\\bifacialvf\data\Output\Tracking_DifferentCities_COMPILED_VF_Albedo0p25_Height0p75.csv"
sazm = 180                  # PV Azimuth(deg) 
slope = 2                  # panel is 2 meter tall module in portrait mode.
C = 0.75                      # GroundClearance(panel slope lengths)
tracking=True; backtrack=True
beta = 10                   # PV tilt (deg)
rowType = "interior"        # RowType(first interior last single)
transFactor = 0.013         # TransmissionFactor(open area fraction)
PVfrontSurface = "glass"    #PVfrontSurface(glass or AR glass)
PVbackSurface = "glass"     # PVbackSurface(glass or AR glass)
offset = 0                  # RefCellOffsetFromBack(panel slope lengths)
albedo = 0.25              # albedo
dataInterval = 60           # DataInterval(minutes)
cellRows = 6                # CellRows(# hor rows in panel)   <-->
D = None

with open (compiledwritefiletitle,'wb') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputheader=['Run', 'Value', 'Front Irr', 'Backk Irr', 'Total Irr Front and Back', 'BackIrradRatio']
            
    sw.writerow(outputheader)

    for ii in range (0, 2):

        if ii==6:
            TMYtoread="..\\bifacialvf\data\\724699TYA.csv"   # Boulder?
            lat = 40.13         # BOULDER
            lng = -105.24           # BOULDER
            tz = -7             # BOULDEr
            city = 'Boulder'

        if ii==6:
            TMYtoread="..\\bifacialvf\data\\722740TYA.csv"   # Tucson a.k.a. T-Town
            lat = 32.13         # Tucson
            lng = -110.95       # T-Town
            tz = -7             # T-Town
            city = 'Tucson'

        if ii==2:
            TMYtoread="..\\bifacialvf\data\\723560TYA.csv"   # Tulsa
            lat = 36.2         # Tulsa
            lng = -95.883       # Tulsa
            tz = -6             # Tulsa
            city = 'Tulsa'

        if ii==3:
            TMYtoread="..\\bifacialvf\data\\723260TYA.csv"   # Knoxville
            lat = 35.817         # Knoxville
            lng = -83.983       # Knoxville
            tz = -5             # Knoxville
            city = 'Knoxville'

        if ii==1:
            TMYtoread="..\\bifacialvf\data\\727930TYA.csv"   # Seattle (Airport)
            lat = 47.467         # Seattle
            lng = -122.317       # Seattle
            tz = -8             # Seattle
            city = 'Seattle'

        if ii==0:
            TMYtoread="..\\bifacialvf\data\\723650TYA.csv"   # Albuquerque 
            lat = 35.04         # Albuquerque
            lng = -106.62       # Albuquerque
            tz = -7             # Albuquerque
            city = 'Albuquerque'

        if ii==6:
            TMYtoread="..\\bifacialvf\data\\725895TYA.csv"   # Klamath
            lat = 42.22         # Klamath
            lng = -121.74       # Klamath
            tz = -8             # Klamath
            city = 'Klamath'
            
        for jj in range (0, 3):

            outputvalues=[]
            
            if jj == 0:
                gcr = 0.25
            if jj == 1:
                gcr = 0.3
            if jj == 2:
                gcr = 0.4            
            
            rtr = 1.0/gcr
            
            writefiletitle="..\\bifacialvf\data\\Output\TRACKING_"+city+"_rtr_"+str(rtr)+".csv"

            bifacialvf.simulate(TMYtoread, writefiletitle, beta, sazm, 
                            C, D, rowType=rowType, transFactor=transFactor, cellRows=cellRows,
                            PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, albedo=albedo, 
                            tracking=tracking, backtrack=backtrack, rtr=rtr)

            #4. Load the results from the resultfile
            (data, metadata) = loadVFresults(writefiletitle)
            #print data.keys()
            # calculate average front and back global tilted irradiance across the module chord
            data['GTIFrontavg'] = data[['No_1_RowFrontGTI', 'No_2_RowFrontGTI','No_3_RowFrontGTI','No_4_RowFrontGTI','No_5_RowFrontGTI','No_6_RowFrontGTI']].mean(axis=1)
            data['GTIBackavg'] = data[['No_1_RowBackGTI', 'No_2_RowBackGTI','No_3_RowBackGTI','No_4_RowBackGTI','No_5_RowBackGTI','No_6_RowBackGTI']].mean(axis=1)

            # Print the annual bifacial ratio
            frontIrrSum = data['GTIFrontavg'].sum()
            backIrrSum = data['GTIBackavg'].sum()
            print('The bifacial ratio for ground clearance {} and rtr spacing {} is: {:.1f}%'.format(C,rtr,backIrrSum/frontIrrSum*100))

            outputvalues.append(city)
            outputvalues.append(gcr)
            outputvalues.append(frontIrrSum)
            outputvalues.append(backIrrSum)
            outputvalues.append(frontIrrSum+backIrrSum)
            outputvalues.append(backIrrSum/frontIrrSum*100)

            sw.writerow(outputvalues)

        print 
        print "Finished with city of " + city
        print "******************************************************************"
        print "******************************************************************"
        print
        
print "FINISHED FINISHED"

 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723650TYA.csv
Location:   "ALBUQUERQUE INTL ARPT [ISIS]"
Lat:  35.04  Long:  -106.62  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  0.75   rtr separation:  4.0   Row type:  interior   Albedo:  0.25
Saving into ..\bifacialvf\data\Output\TRACKING_Albuquerque_rtr_4.0.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  3.0
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Finished
The bifacial ratio for ground clearance 0.75 and rtr spacing 4.0 is: 9.1%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723650TYA.csv
Location:   "ALBUQUERQUE INTL ARPT [ISIS]"
Lat:  35.04  Long:  -106.62  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  0.75   rtr separation:  3.33333333333   Row type:  interior   Albedo:  0.25
Saving into ..\bifacialvf\data\Output\TRACKING_Albuquerque_rtr_3.33333333333.csv
 
 
Dis

In [3]:
# HEIGHT LOOP

compiledwritefiletitle="..\\bifacialvf\data\Output\Tracking_DifferentCities_HEIGHT_Albedo0p4.csv"
sazm = 180                  # PV Azimuth(deg) 
slope = 2                  # panel is 2 meter tall module in portrait mode.
C = 0.50                      # GroundClearance(panel slope lengths)
tracking=True; backtrack=True
beta = 10                   # PV tilt (deg)
rowType = "interior"        # RowType(first interior last single)
transFactor = 0.013         # TransmissionFactor(open area fraction)
PVfrontSurface = "glass"    #PVfrontSurface(glass or AR glass)
PVbackSurface = "glass"     # PVbackSurface(glass or AR glass)
offset = 0                  # RefCellOffsetFromBack(panel slope lengths)
albedo = 0.4               # albedo
dataInterval = 60           # DataInterval(minutes)
cellRows = 6                # CellRows(# hor rows in panel)   <-->
D = None

with open (compiledwritefiletitle,'wb') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputheader=['Run', 'Value', 'Front Irr', 'Backk Irr', 'Total Irr Front and Back', 'BackIrradRatio']
            
    sw.writerow(outputheader)

    for ii in range (0, 7):

        if ii==0:
            TMYtoread="..\\bifacialvf\data\\724699TYA.csv"   # Boulder?
            lat = 40.13         # BOULDER
            lng = -105.24           # BOULDER
            tz = -7             # BOULDEr
            city = 'Boulder'

        if ii==1:
            TMYtoread="..\\bifacialvf\data\\722740TYA.csv"   # Tucson a.k.a. T-Town
            lat = 32.13         # Tucson
            lng = -110.95       # T-Town
            tz = -7             # T-Town
            city = 'Tucson'

        if ii==2:
            TMYtoread="..\\bifacialvf\data\\723560TYA.csv"   # Tulsa
            lat = 36.2         # Tulsa
            lng = -95.883       # Tulsa
            tz = -6             # Tulsa
            city = 'Tulsa'

        if ii==3:
            TMYtoread="..\\bifacialvf\data\\723260TYA.csv"   # Knoxville
            lat = 35.817         # Knoxville
            lng = -83.983       # Knoxville
            tz = -5             # Knoxville
            city = 'Knoxville'

        if ii==4:
            TMYtoread="..\\bifacialvf\data\\727930TYA.csv"   # Seattle (Airport)
            lat = 47.467         # Seattle
            lng = -122.317       # Seattle
            tz = -8             # Seattle
            city = 'Seattle'

        if ii==5:
            TMYtoread="..\\bifacialvf\data\\723650TYA.csv"   # Albuquerque 
            lat = 35.04         # Albuquerque
            lng = -106.62       # Albuquerque
            tz = -7             # Albuquerque
            city = 'Albuquerque'

        if ii==6:
            TMYtoread="..\\bifacialvf\data\\725895TYA.csv"   # Klamath
            lat = 42.22         # Klamath
            lng = -121.74       # Klamath
            tz = -8             # Klamath
            city = 'Klamath'
            
        for jj in range (0, 7):

            outputvalues=[]
            gcr = 0.25
            
            if jj == 0:
                C = 0.5
            if jj == 1:
                C = 0.6
            if jj == 2:
                C = 0.7
            if jj == 3:
                C = 0.8
            if jj == 4:
                C = 0.9
            if jj == 5:
                C = 1.0
            if jj == 6:
                C = 1.2
            
            rtr = 1.0/gcr
            
            writefiletitle="..\\bifacialvf\data\\Output\TRACKING_"+city+"_HEIGHT_"+str(C)+".csv"

            bifacialvf.simulate(TMYtoread, writefiletitle, beta, sazm, 
                            C, D, rowType=rowType, transFactor=transFactor, cellRows=cellRows,
                            PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, albedo=albedo, 
                            tracking=tracking, backtrack=backtrack, rtr=rtr)

            #4. Load the results from the resultfile
            (data, metadata) = loadVFresults(writefiletitle)
            #print data.keys()
            # calculate average front and back global tilted irradiance across the module chord
            data['GTIFrontavg'] = data[['No_1_RowFrontGTI', 'No_2_RowFrontGTI','No_3_RowFrontGTI','No_4_RowFrontGTI','No_5_RowFrontGTI','No_6_RowFrontGTI']].mean(axis=1)
            data['GTIBackavg'] = data[['No_1_RowBackGTI', 'No_2_RowBackGTI','No_3_RowBackGTI','No_4_RowBackGTI','No_5_RowBackGTI','No_6_RowBackGTI']].mean(axis=1)

            # Print the annual bifacial ratio
            frontIrrSum = data['GTIFrontavg'].sum()
            backIrrSum = data['GTIBackavg'].sum()
            print('The bifacial ratio for ground clearance {} and rtr spacing {} is: {:.1f}%'.format(C,rtr,backIrrSum/frontIrrSum*100))

            outputvalues.append(city)
            outputvalues.append(C)
            outputvalues.append(frontIrrSum)
            outputvalues.append(backIrrSum)
            outputvalues.append(frontIrrSum+backIrrSum)
            outputvalues.append(backIrrSum/frontIrrSum*100)

            sw.writerow(outputvalues)

        print 
        print "Finished with city of " + city
        print "******************************************************************"
        print "******************************************************************"
        print
        
print "FINISHED FINISHED"

 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\724699TYA.csv
Location:   "BROOMFIELD/JEFFCO [BOULDER - SURFRAD]"
Lat:  40.13  Long:  -105.24  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  0.5   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\bifacialvf\data\Output\TRACKING_Boulder_HEIGHT_0.5.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  3.0
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Finished
The bifacial ratio for ground clearance 0.5 and rtr spacing 4.0 is: 10.3%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\724699TYA.csv
Location:   "BROOMFIELD/JEFFCO [BOULDER - SURFRAD]"
Lat:  40.13  Long:  -105.24  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  0.6   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\bifacialvf\data\Output\TRACKING_Boulder_HEIGHT_0.6.csv
 
 
Distance be

Finished
The bifacial ratio for ground clearance 1.0 and rtr spacing 4.0 is: 15.2%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\722740TYA.csv
Location:   "TUCSON INTERNATIONAL AP"
Lat:  32.133  Long:  -110.95  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  1.2   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\bifacialvf\data\Output\TRACKING_Tucson_HEIGHT_1.2.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  3.0
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Finished
The bifacial ratio for ground clearance 1.2 and rtr spacing 4.0 is: 16.3%

Finished with city of Tucson
******************************************************************
******************************************************************

 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723560TYA.csv
Location:   "TULSA INTERNATIONAL AIRPORT"
Lat:  36

Finished
The bifacial ratio for ground clearance 0.9 and rtr spacing 4.0 is: 17.3%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723260TYA.csv
Location:   "KNOXVILLE MCGHEE TYSON AP"
Lat:  35.817  Long:  -83.983  Tz  -5.0
Parameters: beta:  0   Sazm:  180   Height:  1.0   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\bifacialvf\data\Output\TRACKING_Knoxville_HEIGHT_1.0.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  3.0
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Finished
The bifacial ratio for ground clearance 1.0 and rtr spacing 4.0 is: 18.1%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723260TYA.csv
Location:   "KNOXVILLE MCGHEE TYSON AP"
Lat:  35.817  Long:  -83.983  Tz  -5.0
Parameters: beta:  0   Sazm:  180   Height:  1.2   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\bifacia

Finished
The bifacial ratio for ground clearance 0.8 and rtr spacing 4.0 is: 13.9%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723650TYA.csv
Location:   "ALBUQUERQUE INTL ARPT [ISIS]"
Lat:  35.04  Long:  -106.62  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  0.9   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\bifacialvf\data\Output\TRACKING_Albuquerque_HEIGHT_0.9.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  3.0
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Finished
The bifacial ratio for ground clearance 0.9 and rtr spacing 4.0 is: 14.7%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\723650TYA.csv
Location:   "ALBUQUERQUE INTL ARPT [ISIS]"
Lat:  35.04  Long:  -106.62  Tz  -7.0
Parameters: beta:  0   Sazm:  180   Height:  1.0   rtr separation:  4.0   Row type:  interior   Albedo:  0.4
Saving into ..\b